In [31]:
import re
import csv
from collections import Counter, defaultdict

In [26]:
file = "sample.log.txt"
def log_file(file):
    with open(file,"r") as f1:
        txt_read = f1.readlines()
    return txt_read
    

    
Ip_address = []

try:
    pattern = r"(\d+\.\d+\.\d+\.\d+)"
    Ip_address = re.findall(pattern, txt_read)  
    if Ip_address:
        print("Extracted IP Addresses:")
        print(Ip_address)
    else:
        print("No IP addresses found in the log file.")
except Exception as e:
    print(f"An error occurred: {e}")

Extracted IP Addresses:
['192.168.1.1', '203.0.113.5', '10.0.0.2', '192.168.1.1', '198.51.100.23', '203.0.113.5', '192.168.1.100', '10.0.0.2', '198.51.100.23', '192.168.1.1', '203.0.113.5', '203.0.113.5', '192.168.1.100', '10.0.0.2', '192.168.1.1', '198.51.100.23', '203.0.113.5', '192.168.1.1', '198.51.100.23', '203.0.113.5', '192.168.1.100', '10.0.0.2', '198.51.100.23', '192.168.1.100', '203.0.113.5', '10.0.0.2', '192.168.1.1', '198.51.100.23', '203.0.113.5', '192.168.1.100', '10.0.0.2', '198.51.100.23', '192.168.1.1', '198.51.100.23']


In [29]:
def count_req_ip(logs):
    ip_counts = {}
    for log in logs:
        ip_counts[log] = ip_counts.get(log,0)+1
    return ip_counts
count_req_ip(Ip_address)
        

{'192.168.1.1': 7,
 '203.0.113.5': 8,
 '10.0.0.2': 6,
 '198.51.100.23': 8,
 '192.168.1.100': 5}

In [33]:
import re
from collections import Counter

# Constants
LOG_FILE = "sample.log.txt"
OUTPUT_FILE = "log_analysis_results.csv"
FAILED_LOGIN_THRESHOLD = 10

# Helper Functions
def read_log_file(file_path):
    """Read the log file and return its content line by line."""
    try:
        with open(file_path, 'r') as file:
            return file.readlines()
    except FileNotFoundError:
        print(f"Error: File {file_path} not found.")
        return []

def extract_requests_per_ip(logs):
    """Extract IP addresses and count their occurrences."""
    ip_pattern = r"(\d+\.\d+\.\d+\.\d+)"
    ip_addresses = [re.match(ip_pattern, log).group(1) for log in logs if re.match(ip_pattern, log)]
    return Counter(ip_addresses)

def find_most_frequent_endpoint(logs):
    """Identify the most frequently accessed endpoint."""
    endpoint_pattern = r'\"[A-Z]+\s+(/[^\s]*)'
    endpoints = [re.search(endpoint_pattern, log).group(1) for log in logs if re.search(endpoint_pattern, log)]
    endpoint_counts = Counter(endpoints)
    return endpoint_counts.most_common(1)[0] if endpoint_counts else ("None", 0)

def detect_failed_logins(logs, threshold):
    """Detect IPs with failed login attempts exceeding a threshold."""
    failed_login_pattern = r"(\d+\.\d+\.\d+\.\d+).*401|Invalid credentials"
    failed_ips = [re.match(failed_login_pattern, log).group(1) for log in logs if re.match(failed_login_pattern, log)]
    failed_counts = Counter(failed_ips)
    return {ip: count for ip, count in failed_counts.items() if count > threshold}

def write_to_csv(ip_data, endpoint_data, suspicious_data, output_file):
    """Write analysis results to a CSV file."""
    with open(output_file, 'w') as file:
        file.write("Requests per IP\n")
        file.write("IP Address,Request Count\n")
        for ip, count in ip_data.items():
            file.write(f"{ip},{count}\n")

        file.write("\nMost Accessed Endpoint\n")
        file.write("Endpoint,Access Count\n")
        file.write(f"{endpoint_data[0]},{endpoint_data[1]}\n")

        file.write("\nSuspicious Activity\n")
        file.write("IP Address,Failed Login Count\n")
        for ip, count in suspicious_data.items():
            file.write(f"{ip},{count}\n")

# Main Function
def main():
    logs = read_log_file(LOG_FILE)
    if not logs:
        return

    # Analyze log file
    requests_per_ip = extract_requests_per_ip(logs)
    most_accessed_endpoint = find_most_frequent_endpoint(logs)
    suspicious_ips = detect_failed_logins(logs, FAILED_LOGIN_THRESHOLD)

    # Display results
    print("Requests per IP Address:")
    for ip, count in requests_per_ip.items():
        print(f"{ip: <20} {count}")

    print("\nMost Frequently Accessed Endpoint:")
    print(f"{most_accessed_endpoint[0]} (Accessed {most_accessed_endpoint[1]} times)")

    print("\nSuspicious Activity Detected:")
    for ip, count in suspicious_ips.items():
        print(f"{ip: <20} {count}")

    # Write results to CSV
    write_to_csv(requests_per_ip, most_accessed_endpoint, suspicious_ips, OUTPUT_FILE)
    print(f"\nResults saved to {OUTPUT_FILE}")

# Execute script
if __name__ == "__main__":
    main()


Requests per IP Address:
192.168.1.1          7
203.0.113.5          8
10.0.0.2             6
198.51.100.23        8
192.168.1.100        5

Most Frequently Accessed Endpoint:
/login (Accessed 13 times)

Suspicious Activity Detected:

Results saved to log_analysis_results.csv


In [34]:
import pandas as pd

In [35]:
df = pd.read_csv("log_analysis_results.csv")

In [36]:
df.T

,IP Address,192.168.1.1,203.0.113.5,10.0.0.2,198.51.100.23,192.168.1.100,Most Accessed Endpoint,Endpoint,/login,Suspicious Activity,IP Address
Requests per IP,Request Count,7,8,6,8,5,NaN,Access Count,13,NaN,Failed Login Count
